In [3]:
// import { ChatOpenAI } from "@langchain/openai";
import { PromptTemplate } from "@langchain/core/prompts";

// const llm = new ChatOpenAI({
//   modelName: process.env.MODEL_NAME,
//   configuration: {
//     baseURL: process.env.BASE_URL,
//     apiKey: process.env.OPEN_API_KEY,
//   },
// });

// const prompt = new PromptTemplate({
//     inputVariables: ['job', 'question'],
//     template: '你是一个{job}，请解释一下{question}'
// })
const prompt = PromptTemplate.fromTemplate('你是一个{job}，请回答一下{question}')

// await prompt.format({ job: '三甲医院的骨科医生', question: '如何预防半月板损伤' })
// const partial = await prompt.partial({ job: '三甲医院的骨科医生' })

const getJob = (hospital: string) => {
    const date = new Date().toLocaleDateString()
    return hospital + '的骨科医生，今天是' + date
}
const partial = await prompt.partial({ job: getJob('三甲医院') })

const rst1 = await partial.format({ question: '如何预防半月板损伤' })
console.log(rst1)

const rst2 = await partial.format({ question: '如何处理半月板损伤' })
console.log(rst2)

你是一个三甲医院的骨科医生，今天是2025/4/25，请回答一下如何预防半月板损伤
你是一个三甲医院的骨科医生，今天是2025/4/25，请回答一下如何处理半月板损伤


In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";

const llm = new ChatOpenAI({
  modelName: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPEN_API_KEY,
  },
});

const humanTemplate = '{humanText}'
const systemTemplate = '你是一个{systemText}'
// const humanMessage = HumanMessagePromptTemplate.fromTemplate(humanTemplate)
// const systemMessage = SystemMessagePromptTemplate.fromTemplate(systemTemplate)
// const prompt = ChatPromptTemplate.fromMessages([systemMessage, humanMessage])

const prompt = ChatPromptTemplate.fromMessages([['system', systemTemplate], ['human', humanTemplate]])

// const message1 = await prompt.format({ humanText: '你好', systemText: '三甲医院的主任医生' })
// console.log('message1---', message1)

// const message2 = await prompt.invoke({ humanText: '你好', systemText: '三甲医院的主任医生' })
// console.log('message2---', message2)

const message3 = await prompt.formatMessages({ humanText: '你好', systemText: '三甲医院的主任医生' })
console.log('message3---', message3)

const outputParse = new StringOutputParser();

// await llm.pipe(outputParse).invoke(message3)

const chain = prompt.pipe(llm).pipe(outputParse);
await chain.invoke({ humanText: '你好', systemText: '三甲医院的主任医生' })

message3--- [
  SystemMessage {
    "content": "你是一个三甲医院的主任医生",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "你好",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]


"您好！很高兴能为您提供医疗相关的帮助。请问您有什么健康方面的问题需要咨询？请注意，虽然我尽力提供准确的信息，但我的建议不能替代专业医疗意见。对于具体病情，请您咨询专业的医疗机构和医务人员。"